In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

In [38]:
def contraction(params,x,p):
    #beta and x are kind of parameters. x is the empirical distribution of x?
    k = int(x.shape[1]/2)
    util1 = np.dot(x[:,0:k], params[0:k])  + params[k]*p[0]
    util2 = np.dot(x[:,k:], params[k+1:2*k+1]) + params[2*k+1]*p[1]
    contr_result = [np.exp(util1)/(1+np.exp(util1)),np.exp(util2)/(1+np.exp(util2))]
    return np.array(contr_result)

#actually caclualte an equilibrium in this game
N=1000
ps = np.array([.5,.5])
betas = np.array([1,-2, 2,-1])

#set up xs
xs = np.random.normal(scale=2, size=(N,2)) #assumes xs are independent? could use a copula?
xs = pd.DataFrame(xs,columns = ['x11','x12'])
#xs.insert(0,'x01',np.ones(N)) # I don't think the model is identified with this
#xs.insert(2,'x02',np.ones(N))
xs = np.array(xs)

print(contraction(betas,xs,ps))

[[0.93671413 0.08144334 0.28680139 ... 0.99824733 0.17175916 0.91664665]
 [0.50754774 0.40846412 0.99957435 ... 0.86957318 0.99959362 0.99029532]]


In [39]:
def contraction_map(betas,x,p):
    """final result is beliefs of firm1/firm2"""
    for i in range(50):
        #print('b1: %.4f, p2: %.4f'%tuple(p))
        p = contraction(betas,x,p).mean(axis=1)
        p = np.flip(p)
        #print('p1: %.4f, p2: %.4f'%tuple(p))
        #print('----- end of iteration %s ----'%i)
    return p

result_ps = contraction_map(betas,xs,ps)
us = np.random.logistic(size=(1000,2))

k = int(xs.shape[1]/2)
es = np.random.logistic(0, 1, (2,N) )
y1 = 1*(np.dot(xs[:,0:k], betas[0:k])  + betas[k]*result_ps[0] + es[0,:] >= 0)
y2 = 1*(np.dot(xs[:,k:], betas[k+1:2*k+1]) + betas[2*k+1]*result_ps[1] + es[1,:] >= 0)
ys = np.array([y1,y2]).transpose()
print(np.flip(result_ps))
print(ys.mean(axis=0))

[0.36869651 0.46321637]
[0.376 0.451]


In [40]:
result_df = np.concatenate( (ys,xs ) ,axis=1)
result_df = pd.DataFrame(result_df, columns=['y1','y2','x11','x12'])
print(result_df)
result_df.to_csv('monte_carlo.csv')

      y1   y2       x11       x12
0    1.0  0.0  3.694716  0.265097
1    0.0  0.0 -1.422896  0.064841
2    1.0  1.0  0.089030  4.130730
3    1.0  1.0  0.260833  3.922238
4    1.0  1.0 -1.493252  1.490628
..   ...  ...       ...       ...
995  1.0  0.0  2.523374  1.203708
996  1.0  1.0  0.979598  3.193566
997  1.0  1.0  7.344859  1.198595
998  1.0  1.0 -0.573211  4.153911
999  1.0  1.0  3.397633  2.562698

[1000 rows x 4 columns]


In [41]:
class BayesNashLogit(GenericLikelihoodModel):
    
    def nloglikeobs(self, params):
        n = self.exog.shape[0]
        k = int(self.exog.shape[1]/2)
        
        p = self.endog.mean(axis=0)
        p = contraction_map(params,self.exog,p)
        
        likelihood = contraction(params,self.exog,p).transpose()
        ll = self.endog*np.log(likelihood) + (1-self.endog)*np.log(1-likelihood)
        
        return -1*ll.sum()
        
        
    
    def fit(self, **kwds):
        """fit the likelihood function using the right start parameters"""
        start_params = np.ones(self.exog.shape[1]+3)
        start_params = np.array(betas)
        return super(BayesNashLogit, self).fit(start_params=start_params,**kwds)

        
        
N = result_df.shape[0]
ys = result_df[['y1','y2']]
xs = result_df[['x11' ,'x12']]


model = BayesNashLogit(ys,xs)
model_fit = model.fit(xtol=1e-12,ftol=1e-12)
print(model_fit.summary(xname=["b11","d1","b12","d2"]))

Optimization terminated successfully.
         Current function value: 0.707240
         Iterations: 167
         Function evaluations: 365
                            BayesNashLogit Results                            
Dep. Variable:           ['y1', 'y2']   Log-Likelihood:                -707.24
Model:                 BayesNashLogit   AIC:                             1418.
Method:            Maximum Likelihood   BIC:                             1428.
Date:                Thu, 01 Apr 2021                                         
Time:                        18:15:13                                         
No. Observations:                1000                                         
Df Residuals:                     998                                         
Df Model:                           1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------

In [44]:
#attempt at standard errors...
H = np.linalg.inv(model.hessian(model_fit.params)/N)
diag = np.diagonal(np.linalg.inv(model.hessian(model_fit.params)))
print(np.sqrt(-1*diag))

[0.06898159 0.21339743 0.12952919 0.29097551]
